<a href="https://colab.research.google.com/github/chaitunalam/idp-parikinsons-disease-detection/blob/main/parkinsons_disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q keras


In [ ]:
!pip install tensorflow


In [ ]:
!pip install keras

In [ ]:
import tensorflow as tf
import keras
from keras.applications.vgg16 import VGG16
from sklearn import svm
from sklearn.metrics import accuracy_score
import os
from keras.applications.vgg16 import VGG16
import tensorflow as tf
from keras import callbacks
from keras import optimizers
from keras.models import Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import load_img
from keras.layers import Input, Lambda, Dense, Flatten,GlobalAveragePooling2D

In [ ]:
!pip install --upgrade tensorflow keras


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data generators with appropriate parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load the training data from the directory
train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/parkinson/Dataset/train',
        batch_size=1,
        target_size=(100, 100),
        class_mode='binary'
)

# Load the testing data from the directory
test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/parkinson/Dataset/test',
        batch_size=1,
        target_size=(100, 100),
        class_mode='binary'
)

# Load the validation data from the directory
val_generator = validation_datagen.flow_from_directory(
        '/content/drive/MyDrive/parkinson/Dataset/valid',
        batch_size=1,
        target_size=(100, 100),
        class_mode='binary'
)

In [ ]:
import cv2
import matplotlib.pyplot as plt
image = cv2.imread("/content/drive/MyDrive/parkinson/Dataset/test/images/NoneDem-9-_jpg.rf.5da58cb6258651e1a070922dee8ad9df.jpg")
plt.imshow(image)
image.shape


In [ ]:
import cv2
import matplotlib.pyplot as plt
image = cv2.imread("/content/drive/MyDrive/parkinson/Dataset/train/images/MildDem-1-_jpg.rf.6bdba105abc128164d6109c86861cfa1.jpg")
plt.imshow(image)
image.shape


In [ ]:
from tensorflow.keras.applications import VGG16

# Build the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False
base_model.summary()



In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)  # You can adjust the number of neurons
predictions = Dense(2, activation='softmax')(x)  # Assuming binary classification

# Create the model
from tensorflow.keras.models import Model
model = Model(inputs=base_model.input, outputs=predictions)



In [ ]:
from keras.applications import VGG16
base_model = VGG16(weights='imagenet', input_shape=(224, 224, 3))
model = Model(inputs = base_model.input, outputs = base_model.get_layer("fc1").output)



In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Import load_img and img_to_array

# Define the training directory path
train_dir = '/content/drive/MyDrive/parkinson/Dataset/train'

# Function to extract features
def extract_features(train_dir, model):
    features = []
    labels = []
    class_names = sorted(os.listdir(train_dir))
    label_dict = {class_name: idx for idx, class_name in enumerate(class_names)}
    for class_name in class_names:
        class_dir = os.path.join(train_dir, class_name)
        for img_path in os.listdir(class_dir):
            img = load_img(os.path.join(class_dir, img_path), target_size=(224, 224))  # Adjust size as needed
            img_array = img_to_array(img)  # Convert image to array
            img_array = np.expand_dims(img_array, axis=0)
            feature = model.predict(img_array)
            features.append(feature)
            labels.append(label_dict[class_name])
    return np.vstack(features), np.array(labels)
train_dir = '/content/drive/MyDrive/parkinson/Dataset/train'  # Ensure this is a valid directory with image files
features, labels = extract_features(train_dir, base_model)
# Extract features and labels
#features, labels = extract_features(train_dir, model)

# Flatten the features if needed
def flatten_features(features):
    return features.reshape(features.shape[0], -1)



features = flatten_features(features)
print("Feature vector dimensions: ",features.shape)


In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Set the path to your dataset directory
dataset_dir = '/content/drive/MyDrive/parkinson/Dataset'

# Image dimensions and batch size
img_width, img_height = 224, 224  # Image size compatible with VGG16
batch_size = 32

# Data augmentation and rescaling
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load training and validation datasets
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Feature Extraction using VGG16
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Extract features and flatten them
def extract_features(generator, model, batch_size):
    features = []
    labels = []
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features.append(features_batch)
        labels.append(labels_batch)
        if len(features) * batch_size >= generator.samples:
            break
    features = np.vstack(features)
    labels = np.hstack(labels)
    return features, labels

# Extract features for training and validation data
train_features, train_labels = extract_features(train_generator, vgg16_model, batch_size)
validation_features, validation_labels = extract_features(validation_generator, vgg16_model, batch_size)

# Reshape the features to fit classifiers (flattening)
train_features = train_features.reshape(train_features.shape[0], -1)
validation_features = validation_features.reshape(validation_features.shape[0], -1)

# Split training data for train-test split
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

# Train SVM Classifier
svm_clf = SVC(kernel='linear', C=1)  # You can experiment with different kernels
svm_clf.fit(X_train, y_train)

# Predict and evaluate SVM
y_pred_svm = svm_clf.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_pred_svm)
svm_cm = confusion_matrix(y_test, y_pred_svm)

print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")
print("SVM Confusion Matrix:")
print(svm_cm)

# Train Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

# Predict and evaluate Decision Tree
y_pred_dt = dt_clf.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_pred_dt)
dt_cm = confusion_matrix(y_test, y_pred_dt)

print(f"Decision Tree Accuracy: {dt_accuracy * 100:.2f}%")
print("Decision Tree Confusion Matrix:")
print(dt_cm)

# Train KNN Classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn_clf.fit(X_train, y_train)

# Predict and evaluate KNN
y_pred_knn = knn_clf.predict(X_test)
knn_accuracy = accuracy_score(y_test, y_pred_knn)
knn_cm = confusion_matrix(y_test, y_pred_knn)

print(f"KNN Accuracy: {knn_accuracy * 100:.2f}%")
print("KNN Confusion Matrix:")
print(knn_cm)




In [ ]:
# Build and Train ANN Classifier
ann_model = Sequential()
ann_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
ann_model.add(Dropout(0.5))
ann_model.add(Dense(64, activation='relu'))
ann_model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the ANN
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Predict and evaluate ANN
y_pred_ann = (ann_model.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary
ann_accuracy = accuracy_score(y_test, y_pred_ann)
ann_cm = confusion_matrix(y_test, y_pred_ann)

print(f"ANN Accuracy: {ann_accuracy * 100:.2f}%")
print("ANN Confusion Matrix:")
print(ann_cm)
# ANN Confusion Matrix
sns.heatmap(cm_ann, annot=True, fmt='d', cmap='Purples', ax=axes[1, 1],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[1, 1].set_title('ANN Confusion Matrix', fontsize=16)
axes[1, 1].set_xlabel('Predicted Label', fontsize=14)
axes[1, 1].set_ylabel('True Label', fontsize=14)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Assuming you already have X_train, X_test, y_train, and y_test defined

# Train SVM Classifier
svm_clf = SVC(kernel='linear', C=1)
svm_clf.fit(X_train, y_train)

# Predict and evaluate SVM
y_pred_svm = svm_clf.predict(X_test)
cm_svm = confusion_matrix(y_test, y_pred_svm)

# Train Decision Tree Classifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

# Predict and evaluate Decision Tree
y_pred_tree = decision_tree.predict(X_test)
cm_tree = confusion_matrix(y_test, y_pred_tree)

# Train KNN Classifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# Predict and evaluate KNN
y_pred_knn = knn.predict(X_test)
cm_knn = confusion_matrix(y_test, y_pred_knn)

# Train ANN Classifier
ann_model = Sequential()
ann_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
ann_model.add(Dense(1, activation='sigmoid'))  # Use 'softmax' for multi-class
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Predictions and evaluation for ANN
y_pred_ann = ann_model.predict(X_test)
y_pred_ann = (y_pred_ann > 0.5).astype(int)  # Convert probabilities to binary
cm_ann = confusion_matrix(y_test, y_pred_ann)

# Plotting the confusion matrices
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# SVM Confusion Matrix
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Purples', ax=axes[0, 0],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[0, 0].set_title('SVM Confusion Matrix', fontsize=16)
axes[0, 0].set_xlabel('Predicted Label', fontsize=14)
axes[0, 0].set_ylabel('True Label', fontsize=14)

# Decision Tree Confusion Matrix
sns.heatmap(cm_tree, annot=True, fmt='d', cmap='Purples', ax=axes[0, 1],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[0, 1].set_title('Decision Tree Confusion Matrix', fontsize=16)
axes[0, 1].set_xlabel('Predicted Label', fontsize=14)
axes[0, 1].set_ylabel('True Label', fontsize=14)

# KNN Confusion Matrix
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Purples', ax=axes[1, 0],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[1, 0].set_title('KNN Confusion Matrix', fontsize=16)
axes[1, 0].set_xlabel('Predicted Label', fontsize=14)
axes[1, 0].set_ylabel('True Label', fontsize=14)

# ANN Confusion Matrix
sns.heatmap(cm_ann, annot=True, fmt='d', cmap='Purples', ax=axes[1, 1],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[1, 1].set_title('ANN Confusion Matrix', fontsize=16)
axes[1, 1].set_xlabel('Predicted Label', fontsize=14)
axes[1, 1].set_ylabel('True Label', fontsize=14)

plt.tight_layout()
plt.show()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix

# Sample accuracy values
accuracies = {
    'SVM': 93.19,
    'Decision Tree': 92.86,
    'KNN': 96.51,
    'ANN': 94.19
}

# Sample confusion matrices
confusion_matrices = {
    'SVM': np.array([[0, 1, 0],
                     [0, 555, 25],
                     [0, 15, 6]]),
    'Decision Tree': np.array([[0, 1, 0],
                                [0, 555, 25],
                                [1, 16, 4]]),
    'KNN': np.array([[0, 1, 0],
                     [0, 578, 2],
                     [0, 18, 3]]),
    'ANN': np.array([[0, 1, 0],
                     [0, 567, 0],
                     [0, 34, 0]])
}

# Plotting Confusion Matrices
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for i, (clf_name, cm) in enumerate(confusion_matrices.items()):
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                cbar=True, xticklabels=['0', '1', '2'], yticklabels=['0', '1', '2'])
    axes[i].set_title(f'{clf_name} Confusion Matrix', fontsize=16)
    axes[i].set_xlabel('Predicted', fontsize=12)
    axes[i].set_ylabel('True', fontsize=12)

plt.tight_layout()
plt.show()

# Plotting Accuracy Comparison
plt.figure(figsize=(10, 6))
bars = plt.bar(accuracies.keys(), accuracies.values(), color=['purple', 'orange', 'green', 'blue'])

# Adding accuracy values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, f"{yval:.2f}%", va='bottom', ha='center')  # va: vertical alignment

plt.title('Classifier Accuracy Comparison', fontsize=16)
plt.ylabel('Accuracy (%)', fontsize=14)
plt.ylim(90, 100)  # Set the y-axis limit from 90 to 100 for better visualization
plt.axhline(y=95, color='r', linestyle='--', label='95% Threshold')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()




In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Set the path to your dataset directory
dataset_dir = '/content/drive/MyDrive/parkinson/Dataset'

# Image dimensions and batch size
img_width, img_height = 224, 224  # Image size compatible with VGG16
batch_size = 32

# Data augmentation and rescaling
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load training and validation datasets
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Feature Extraction using VGG19
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Extract features and flatten them
def extract_features(generator, model, batch_size):
    features = []
    labels = []
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features.append(features_batch)
        labels.append(labels_batch)
        if len(features) * batch_size >= generator.samples:
            break
    features = np.vstack(features)
    labels = np.hstack(labels)
    return features, labels

# Extract features for training and validation data
train_features, train_labels = extract_features(train_generator, vgg19_model, batch_size)
validation_features, validation_labels = extract_features(validation_generator, vgg19_model, batch_size)

# Reshape the features to fit classifiers (flattening)
train_features = train_features.reshape(train_features.shape[0], -1)
validation_features = validation_features.reshape(validation_features.shape[0], -1)

# Split training data for train-test split
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

# Train SVM Classifier
svm_clf = SVC(kernel='linear', C=1)  # You can experiment with different kernels
svm_clf.fit(X_train, y_train)

# Predict and evaluate SVM
y_pred_svm = svm_clf.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_pred_svm)
svm_cm = confusion_matrix(y_test, y_pred_svm)

print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")
print("SVM Confusion Matrix:")
print(svm_cm)

# Train Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

# Predict and evaluate Decision Tree
y_pred_dt = dt_clf.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_pred_dt)
dt_cm = confusion_matrix(y_test, y_pred_dt)

print(f"Decision Tree Accuracy: {dt_accuracy * 100:.2f}%")
print("Decision Tree Confusion Matrix:")
print(dt_cm)

# Train KNN Classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn_clf.fit(X_train, y_train)

# Predict and evaluate KNN
y_pred_knn = knn_clf.predict(X_test)
knn_accuracy = accuracy_score(y_test, y_pred_knn)
knn_cm = confusion_matrix(y_test, y_pred_knn)

print(f"KNN Accuracy: {knn_accuracy * 100:.2f}%")
print("KNN Confusion Matrix:")
print(knn_cm)









In [ ]:
# Build and Train ANN Classifier
ann_model = Sequential()
ann_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
ann_model.add(Dropout(0.5))
ann_model.add(Dense(64, activation='relu'))
ann_model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the ANN
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Predict and evaluate ANN
y_pred_ann = (ann_model.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary
ann_accuracy = accuracy_score(y_test, y_pred_ann)
ann_cm = confusion_matrix(y_test, y_pred_ann)

print(f"ANN Accuracy: {ann_accuracy * 100:.2f}%")
print("ANN Confusion Matrix:")
print(ann_cm)
# ANN Confusion Matrix
sns.heatmap(cm_ann, annot=True, fmt='d', cmap='Purples', ax=axes[1, 1],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[1, 1].set_title('ANN Confusion Matrix', fontsize=16)
axes[1, 1].set_xlabel('Predicted Label', fontsize=14)
axes[1, 1].set_ylabel('True Label', fontsize=14)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Updated accuracy values
accuracies = {
    'SVM': 94.85,
    'Decision Tree': 92.36,
    'KNN': 96.35,
    'ANN': 94.19
}

# Updated confusion matrices
confusion_matrices = {
    'SVM': np.array([[0, 2, 0],
                     [0, 564, 12],
                     [0, 17, 7]]),
    'Decision Tree': np.array([[0, 2, 0],
                               [0, 552, 24],
                               [0, 20, 4]]),
    'KNN': np.array([[0, 2, 0],
                     [0, 576, 0],
                     [0, 20, 4]]),
    'ANN': np.array([[0, 1, 0],
                     [0, 567, 0],
                     [0, 34, 0]])
}

# Plotting Confusion Matrices
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for i, (clf_name, cm) in enumerate(confusion_matrices.items()):
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                cbar=True, xticklabels=['0', '1', '2'], yticklabels=['0', '1', '2'])
    axes[i].set_title(f'{clf_name} Confusion Matrix', fontsize=16)
    axes[i].set_xlabel('Predicted', fontsize=12)
    axes[i].set_ylabel('True', fontsize=12)

plt.tight_layout()
plt.show()

# Plotting Accuracy Comparison
plt.figure(figsize=(10, 6))
bars = plt.bar(accuracies.keys(), accuracies.values(), color=['purple', 'orange', 'green', 'blue'])

# Adding accuracy values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, f"{yval:.2f}%", va='bottom', ha='center')  # va: vertical alignment

plt.title('Classifier Accuracy Comparison', fontsize=16)
plt.ylabel('Accuracy (%)', fontsize=14)
plt.ylim(90, 100)  # Set the y-axis limit from 90 to 100 for better visualization
plt.axhline(y=95, color='r', linestyle='--', label='95% Threshold')  # Add threshold line at 95%
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()




In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Set the path to your dataset directory
dataset_dir = '/content/drive/MyDrive/parkinson/Dataset'

# Image dimensions and batch size
img_width, img_height = 224, 224  # Image size compatible with ResNet50
batch_size = 32

# Data augmentation and rescaling
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load training and validation datasets
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Feature Extraction using ResNet50
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Extract features and flatten them
def extract_features(generator, model, batch_size):
    features = []
    labels = []
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features.append(features_batch)
        labels.append(labels_batch)
        if len(features) * batch_size >= generator.samples:
            break
    features = np.vstack(features)
    labels = np.hstack(labels)
    return features, labels

# Extract features for training and validation data
train_features, train_labels = extract_features(train_generator, resnet50_model, batch_size)
validation_features, validation_labels = extract_features(validation_generator, resnet50_model, batch_size)

# Reshape the features to fit classifiers (flattening)
train_features = train_features.reshape(train_features.shape[0], -1)
validation_features = validation_features.reshape(validation_features.shape[0], -1)

# Split training data for train-test split
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

# Train SVM Classifier
svm_clf = SVC(kernel='linear', C=1)  # You can experiment with different kernels
svm_clf.fit(X_train, y_train)

# Predict and evaluate SVM
y_pred_svm = svm_clf.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_pred_svm)
svm_cm = confusion_matrix(y_test, y_pred_svm)

print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")
print("SVM Confusion Matrix:")
print(svm_cm)

# Train Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

# Predict and evaluate Decision Tree
y_pred_dt = dt_clf.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_pred_dt)
dt_cm = confusion_matrix(y_test, y_pred_dt)

print(f"Decision Tree Accuracy: {dt_accuracy * 100:.2f}%")
print("Decision Tree Confusion Matrix:")
print(dt_cm)

# Train KNN Classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn_clf.fit(X_train, y_train)

# Predict and evaluate KNN
y_pred_knn = knn_clf.predict(X_test)
knn_accuracy = accuracy_score(y_test, y_pred_knn)
knn_cm = confusion_matrix(y_test, y_pred_knn)

print(f"KNN Accuracy: {knn_accuracy * 100:.2f}%")
print("KNN Confusion Matrix:")
print(knn_cm)

# Build and Train ANN Classifier
ann_model = Sequential()
ann_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
ann_model.add(Dropout(0.5))
ann_model.add(Dense(64, activation='relu'))
ann_model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the ANN
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Predict and evaluate ANN
y_pred_ann = (ann_model.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary
ann_accuracy = accuracy_score(y_test, y_pred_ann)
ann_cm = confusion_matrix(y_test, y_pred_ann)

print(f"ANN Accuracy: {ann_accuracy * 100:.2f}%")
print("ANN Confusion Matrix:")
print(ann_cm)

# Plotting all confusion matrices
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

# SVM Confusion Matrix
sns.heatmap(svm_cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[0].set_title('SVM Confusion Matrix', fontsize=16)
axes[0].set_xlabel('Predicted Label', fontsize=14)
axes[0].set_ylabel('True Label', fontsize=14)

# Decision Tree Confusion Matrix
sns.heatmap(dt_cm, annot=True, fmt='d', cmap='Oranges', ax=axes[1],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[1].set_title('Decision Tree Confusion Matrix', fontsize=16)
axes[1].set_xlabel('Predicted Label', fontsize=14)
axes[1].set_ylabel('True Label', fontsize=14)

# KNN Confusion Matrix
sns.heatmap(knn_cm, annot=True, fmt='d', cmap='Greens', ax=axes[2],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[2].set_title('KNN Confusion Matrix', fontsize=16)
axes[2].set_xlabel('Predicted Label', fontsize=14)
axes[2].set_ylabel('True Label', fontsize=14)

# ANN Confusion Matrix (Fixed the variable name)
sns.heatmap(ann_cm, annot=True, fmt='d', cmap='Purples', ax=axes[3],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[3].set_title('ANN Confusion Matrix', fontsize=16)
axes[3].set_xlabel('Predicted Label', fontsize=14)
axes[3].set_ylabel('True Label', fontsize=14)

plt.tight_layout()
plt.show()




In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Set the path to your dataset directory
dataset_dir = '/content/drive/MyDrive/parkinson/Dataset'

# Image dimensions and batch size
img_width, img_height = 224, 224  # Image size compatible with ResNet50
batch_size = 32

# Data augmentation and rescaling
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load training and validation datasets
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Feature Extraction using ResNet50
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Extract features and flatten them
def extract_features(generator, model, batch_size):
    features = []
    labels = []
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features.append(features_batch)
        labels.append(labels_batch)
        if len(features) * batch_size >= generator.samples:
            break
    features = np.vstack(features)
    labels = np.hstack(labels)
    return features, labels

# Extract features for training and validation data
train_features, train_labels = extract_features(train_generator, resnet50_model, batch_size)
validation_features, validation_labels = extract_features(validation_generator, resnet50_model, batch_size)

# Reshape the features to fit classifiers (flattening)
train_features = train_features.reshape(train_features.shape[0], -1)
validation_features = validation_features.reshape(validation_features.shape[0], -1)

# Split training data for train-test split
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

# Train SVM Classifier
svm_clf = SVC(kernel='linear', C=1)
svm_clf.fit(X_train, y_train)

# Predict and evaluate SVM
y_pred_svm = svm_clf.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_pred_svm)
svm_cm = confusion_matrix(y_test, y_pred_svm)

print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")
print("SVM Confusion Matrix:")
print(svm_cm)
print(classification_report(y_test, y_pred_svm))

# Train Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

# Predict and evaluate Decision Tree
y_pred_dt = dt_clf.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_pred_dt)
dt_cm = confusion_matrix(y_test, y_pred_dt)

print(f"Decision Tree Accuracy: {dt_accuracy * 100:.2f}%")
print("Decision Tree Confusion Matrix:")
print(dt_cm)
print(classification_report(y_test, y_pred_dt))

# Train KNN Classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)

# Predict and evaluate KNN
y_pred_knn = knn_clf.predict(X_test)
knn_accuracy = accuracy_score(y_test, y_pred_knn)
knn_cm = confusion_matrix(y_test, y_pred_knn)

print(f"KNN Accuracy: {knn_accuracy * 100:.2f}%")
print("KNN Confusion Matrix:")
print(knn_cm)
print(classification_report(y_test, y_pred_knn))

# Build and Train ANN Classifier
ann_model = Sequential()
ann_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
ann_model.add(Dropout(0.5))
ann_model.add(Dense(64, activation='relu'))
ann_model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the ANN
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Predict and evaluate ANN
y_pred_ann = (ann_model.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary
ann_accuracy = accuracy_score(y_test, y_pred_ann)
ann_cm = confusion_matrix(y_test, y_pred_ann)

print(f"ANN Accuracy: {ann_accuracy * 100:.2f}%")
print("ANN Confusion Matrix:")
print(ann_cm)
print(classification_report(y_test, y_pred_ann))

# Plotting all confusion matrices
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

# SVM Confusion Matrix
sns.heatmap(svm_cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[0].set_title('SVM Confusion Matrix', fontsize=16)
axes[0].set_xlabel('Predicted Label', fontsize=14)
axes[0].set_ylabel('True Label', fontsize=14)

# Decision Tree Confusion Matrix
sns.heatmap(dt_cm, annot=True, fmt='d', cmap='Oranges', ax=axes[1],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[1].set_title('Decision Tree Confusion Matrix', fontsize=16)
axes[1].set_xlabel('Predicted Label', fontsize=14)
axes[1].set_ylabel('True Label', fontsize=14)

# KNN Confusion Matrix
sns.heatmap(knn_cm, annot=True, fmt='d', cmap='Greens', ax=axes[2],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[2].set_title('KNN Confusion Matrix', fontsize=16)
axes[2].set_xlabel('Predicted Label', fontsize=14)
axes[2].set_ylabel('True Label', fontsize=14)

# ANN Confusion Matrix
sns.heatmap(ann_cm, annot=True, fmt='d', cmap='Purples', ax=axes[3],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[3].set_title('ANN Confusion Matrix', fontsize=16)
axes[3].set_xlabel('Predicted Label', fontsize=14)
axes[3].set_ylabel('True Label', fontsize=14)

plt.tight_layout()
plt.show()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Updated accuracy values
accuracies = {
    'SVM': 94.35,
    'Decision Tree': 93.19,
    'KNN': 96.68,
    'ANN': 96.35
}

# Updated confusion matrices
confusion_matrices = {
    'SVM': np.array([[0, 1, 0],
                     [0, 562, 18],
                     [0, 15, 6]]),
    'Decision Tree': np.array([[0, 1, 0],
                               [0, 554, 26],
                               [0, 14, 7]]),
    'KNN': np.array([[0, 1, 0],
                     [0, 578, 2],
                     [0, 17, 4]]),
    'ANN': np.array([[0, 1, 0],
                     [0, 580, 0],
                     [0, 21, 0]])
}

# Plotting Confusion Matrices
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for i, (clf_name, cm) in enumerate(confusion_matrices.items()):
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                cbar=True, xticklabels=['0', '1', '2'], yticklabels=['0', '1', '2'])
    axes[i].set_title(f'{clf_name} Confusion Matrix', fontsize=16)
    axes[i].set_xlabel('Predicted', fontsize=12)
    axes[i].set_ylabel('True', fontsize=12)

plt.tight_layout()
plt.show()

# Plotting Accuracy Comparison
plt.figure(figsize=(10, 6))
bars = plt.bar(accuracies.keys(), accuracies.values(), color=['purple', 'orange', 'green', 'blue'])

# Adding accuracy values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, f"{yval:.2f}%", va='bottom', ha='center')

plt.title('Classifier Accuracy Comparison', fontsize=16)
plt.ylabel('Accuracy (%)', fontsize=14)
plt.ylim(90, 100)  # Set the y-axis limit from 90 to 100 for better visualization
plt.axhline(y=95, color='r', linestyle='--', label='95% Threshold')  # Add threshold line at 95%
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()




In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Set the path to your dataset directory
dataset_dir = '/content/drive/MyDrive/parkinson/Dataset'

# Image dimensions and batch size
img_width, img_height = 224, 224  # Image size compatible with VGG16
batch_size = 32

# Data augmentation and rescaling
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load training and validation datasets
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Feature Extraction using VGG19
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Extract features and flatten them
def extract_features(generator, model, batch_size):
    features = []
    labels = []
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features.append(features_batch)
        labels.append(labels_batch)
        if len(features) * batch_size >= generator.samples:
            break
    features = np.vstack(features)
    labels = np.hstack(labels)
    return features, labels

# Extract features for training and validation data
train_features, train_labels = extract_features(train_generator, vgg19_model, batch_size)
validation_features, validation_labels = extract_features(validation_generator, vgg19_model, batch_size)

# Reshape the features to fit classifiers (flattening)
train_features = train_features.reshape(train_features.shape[0], -1)
validation_features = validation_features.reshape(validation_features.shape[0], -1)

# Split training data for train-test split
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

# Train SVM Classifier
svm_clf = SVC(kernel='linear', C=1)  # You can experiment with different kernels
svm_clf.fit(X_train, y_train)

# Predict and evaluate SVM
y_pred_svm = svm_clf.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_pred_svm)
svm_cm = confusion_matrix(y_test, y_pred_svm)

print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")
print("SVM Confusion Matrix:")
print(svm_cm)

# Train Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

# Predict and evaluate Decision Tree
y_pred_dt = dt_clf.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_pred_dt)
dt_cm = confusion_matrix(y_test, y_pred_dt)

print(f"Decision Tree Accuracy: {dt_accuracy * 100:.2f}%")
print("Decision Tree Confusion Matrix:")
print(dt_cm)

# Train KNN Classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn_clf.fit(X_train, y_train)

# Predict and evaluate KNN
y_pred_knn = knn_clf.predict(X_test)
knn_accuracy = accuracy_score(y_test, y_pred_knn)
knn_cm = confusion_matrix(y_test, y_pred_knn)

print(f"KNN Accuracy: {knn_accuracy * 100:.2f}%")
print("KNN Confusion Matrix:")
print(knn_cm)









In [ ]:
# Build and Train ANN Classifier
ann_model = Sequential()
ann_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
ann_model.add(Dropout(0.5))
ann_model.add(Dense(64, activation='relu'))
ann_model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the ANN
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Predict and evaluate ANN
y_pred_ann = (ann_model.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary
ann_accuracy = accuracy_score(y_test, y_pred_ann)
ann_cm = confusion_matrix(y_test, y_pred_ann)

print(f"ANN Accuracy: {ann_accuracy * 100:.2f}%")
print("ANN Confusion Matrix:")
print(ann_cm)
# ANN Confusion Matrix
sns.heatmap(cm_ann, annot=True, fmt='d', cmap='Purples', ax=axes[1, 1],
            cbar=True, xticklabels=['0', '1'], yticklabels=['0', '1'], linewidths=0.5)
axes[1, 1].set_title('ANN Confusion Matrix', fontsize=16)
axes[1, 1].set_xlabel('Predicted Label', fontsize=14)
axes[1, 1].set_ylabel('True Label', fontsize=14)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Updated accuracy values
accuracies = {
    'SVM': 94.85,
    'Decision Tree': 92.36,
    'KNN': 96.35,
    'ANN': 94.19
}

# Updated confusion matrices
confusion_matrices = {
    'SVM': np.array([[0, 2, 0],
                     [0, 564, 12],
                     [0, 17, 7]]),
    'Decision Tree': np.array([[0, 2, 0],
                               [0, 552, 24],
                               [0, 20, 4]]),
    'KNN': np.array([[0, 2, 0],
                     [0, 576, 0],
                     [0, 20, 4]]),
    'ANN': np.array([[0, 1, 0],
                     [0, 567, 0],
                     [0, 34, 0]])
}

# Plotting Confusion Matrices
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for i, (clf_name, cm) in enumerate(confusion_matrices.items()):
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                cbar=True, xticklabels=['0', '1', '2'], yticklabels=['0', '1', '2'])
    axes[i].set_title(f'{clf_name} Confusion Matrix', fontsize=16)
    axes[i].set_xlabel('Predicted', fontsize=12)
    axes[i].set_ylabel('True', fontsize=12)

plt.tight_layout()
plt.show()

# Plotting Accuracy Comparison
plt.figure(figsize=(10, 6))
bars = plt.bar(accuracies.keys(), accuracies.values(), color=['purple', 'orange', 'green', 'blue'])

# Adding accuracy values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, f"{yval:.2f}%", va='bottom', ha='center')  # va: vertical alignment

plt.title('Classifier Accuracy Comparison', fontsize=16)
plt.ylabel('Accuracy (%)', fontsize=14)
plt.ylim(90, 100)  # Set the y-axis limit from 90 to 100 for better visualization
plt.axhline(y=95, color='r', linestyle='--', label='95% Threshold')  # Add threshold line at 95%
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

